In [2]:
from google.colab import files
uploaded = files.upload()

In [21]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [22]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
app_df = application_df.drop(columns=[ 'EIN'])
app_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [24]:
# Look at NAME counts for binning
countsA = app_df['NAME'].value_counts()
countsA

PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: NAME, Length: 19568, dtype: int64

In [25]:

# Choose a cutoff value and create a list of names_replace to be replaced
# use the variable name names_replace
names_replace = list(countsA[countsA<10].index)
names_replace

['HEALTH CARE FOR ALL-CALIFORNIA',
 'PHI ETA SIGMA NATIONAL HONOR SOCIETY INC',
 'RECREATION CENTERS OF SUN CITY INC',
 'ASSOCIATION OF SCHOOL BOOSTER CLUBS',
 'INTERNATIONAL SOCIETY DAUGHTERS OF UTAH PIONEERS',
 'SOUTHERN OFF ROAD BICYCLE ASSOCIATION',
 'OREGON EDUCATION ASSOCIATION',
 'NATIONAL CHRISTIAN CHARITABLE FOUNDATION INC',
 'NATIONAL COUNCIL OF NEGRO WOMEN INC',
 'INDEPENDENT ELECTRICAL CONTRACTORS INC',
 'PAGAN PRIDE PROJECT INC',
 'TUSKEGEE AIRMEN INC',
 'ALPHA PHI OMEGA',
 'SEATTLE CHILDRENS HOSPITAL',
 'FBI NATIONAL CITIZENS ACADEMY ALUMNI ASSOCIATION INC',
 'UNITED OSTOMY ASSOCIATIONS OF AMERICA INC',
 'AMERICAN CHEMICAL SOCIETY',
 'MILITARY ORDER OF THE WORLD WARS',
 'PTA FLORIDA CONGRESS',
 'NATIONAL LEAGUE OF YOUNG MEN INC',
 'AMERICAN HIBISCUS',
 'AMERICAN VOLKSSPORT ASSOCIATION INC',
 'THETA NU XI MULTICULTURAL SORORITY INC',
 'QUOTA CLUB INTERNATIONAL INC',
 'AMERICAN INSTITUTE OF GRAPHIC ARTS',
 'AFRICAN-AMERICAN POSTAL LEAGUE UNITED FOR SUCCESS A-PLUS',
 'SHRINE

In [26]:
# Replace in dataframe
for name in names_replace:
    app_df['NAME'] = app_df['NAME'].replace(name,"Other")

# Check to make sure binning was successful
app_df['NAME'].value_counts()
     

Other                                      21022
PARENT BOOSTER USA INC                      1260
TOPS CLUB INC                                765
UNITED STATES BOWLING CONGRESS INC           700
WASHINGTON STATE UNIVERSITY                  492
                                           ...  
CASCADE 4-H FOUNDATION                        10
FREE & ACCEPTED MASONS OF WASHINGTON          10
NEW MEXICO GARDEN CLUBS INC                   10
NATIONAL ASSOCIATION OF HISPANIC NURSES       10
UNION OF CALIFORNIA STATE WORKERS             10
Name: NAME, Length: 223, dtype: int64

In [27]:
# Determine the number of unique values in each column.
app_df.nunique()

NAME                       223
APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [31]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = application_df['APPLICATION_TYPE'].value_counts()
app_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [32]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_counts[app_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    app_df['APPLICATION_TYPE'] = app_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
app_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [33]:
# Look at CLASSIFICATION value counts for binning
class_counts = app_df['CLASSIFICATION'].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [34]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts[class_counts > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [35]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_counts[class_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [36]:
# Convert categorical data to numeric with `pd.get_dummies`
app_df = pd.get_dummies(app_df)
app_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,NAME_ALABAMA TREASURE FOREST ASSOCIATION,NAME_ALPHA PHI SIGMA,NAME_ALPHA PHI SIGMA INC,NAME_ALTRUSA INTERNATIONAL FOUNDATION INC,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [37]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
x = app_df.drop(columns=['IS_SUCCESSFUL']).values
y = app_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

In [39]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
print(X_train_scaled[0])

[ 1.27279628e-02 -3.10230459e-02 -2.05258837e-02 -5.40746855e-02
 -3.41698725e-02 -3.06649185e-02 -9.65328854e-02 -2.48169963e-02
 -2.95876597e-02 -3.98732580e-02 -1.09841605e-01 -1.88804550e-02
 -7.65852058e-02 -3.55668613e-02 -2.48169963e-02 -2.54621137e-02
 -1.70774607e-02 -2.90340999e-02 -1.97203116e-02 -1.80015158e-02
 -3.11896488e-02 -2.48169963e-02 -3.01310863e-02 -2.73064027e-02
 -1.88804550e-02 -2.67057558e-02 -1.70774607e-02 -3.77811172e-02
 -2.05258837e-02 -1.88804550e-02 -3.46417652e-02 -1.70774607e-02
 -1.97203116e-02 -2.48169963e-02 -1.70774607e-02 -2.13010613e-02
 -2.20490505e-02 -2.60913254e-02 -1.97203116e-02 -2.34737675e-02
 -2.41546990e-02 -2.20490505e-02 -3.17057291e-02 -2.05258837e-02
 -2.54621137e-02 -1.97203116e-02 -1.70774607e-02 -1.97203116e-02
 -1.80015158e-02 -4.80140032e-02 -2.48169963e-02 -2.60913254e-02
 -2.60913254e-02 -2.67057558e-02 -1.50604166e-02 -1.70774607e-02
 -1.39430105e-02 -1.97203116e-02 -4.02787265e-02 -6.27312944e-02
 -3.51073455e-02 -4.26311

## Part2: Compile, Train and Evaluate the Model

In [40]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input_features = len(X_train[0])
layer_1_nodes = 100
layer_2_nodes = 80


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=layer_1_nodes, 
                          input_dim=num_input_features, 
                          activation='relu'))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=layer_2_nodes,
                          activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1,
                          activation='sigmoid'))

# Check the structure of the model
nn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               33200     
                                                                 
 dense_1 (Dense)             (None, 80)                8080      
                                                                 
 dense_2 (Dense)             (None, 1)                 81        
                                                                 
Total params: 41,361
Trainable params: 41,361
Non-trainable params: 0
_________________________________________________________________


In [41]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [42]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, validation_split=0.15, epochs=100, batch_size=10)

Epoch 1/100
2624/2624 [==============================] - 8s 3ms/step - loss: 0.4726 - accuracy: 0.7751 - val_loss: 0.4591 - val_accuracy: 0.7826
Epoch 2/100
2624/2624 [==============================] - 6s 2ms/step - loss: 0.4438 - accuracy: 0.7861 - val_loss: 0.4532 - val_accuracy: 0.7828
Epoch 3/100
2624/2624 [==============================] - 6s 2ms/step - loss: 0.4394 - accuracy: 0.7868 - val_loss: 0.4506 - val_accuracy: 0.7858
Epoch 4/100
2624/2624 [==============================] - 7s 3ms/step - loss: 0.4376 - accuracy: 0.7871 - val_loss: 0.4510 - val_accuracy: 0.7849
Epoch 5/100
2624/2624 [==============================] - 6s 2ms/step - loss: 0.4352 - accuracy: 0.7894 - val_loss: 0.4606 - val_accuracy: 0.7810
Epoch 6/100
2624/2624 [==============================] - 6s 2ms/step - loss: 0.4350 - accuracy: 0.7885 - val_loss: 0.4545 - val_accuracy: 0.7834
Epoch 7/100
2624/2624 [==============================] - 11s 4ms/step - loss: 0.4336 - accuracy: 0.7907 - val_loss: 0.4538 - val_a

In [43]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

108/108 - 0s - loss: 0.7812 - accuracy: 0.7749 - 137ms/epoch - 1ms/step
Loss: 0.7811647057533264, Accuracy: 0.7749271392822266


In [44]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')

In [45]:
# Export our model to HDF5 file
from google.colab import files

nn.save('/content/AlphabetSoupCharity_Optimization.h5')
files.download('/content/AlphabetSoupCharity_Optimization.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')